Ao aplicar Análise Exploratória de Dados vou responder a estas 10 perguntas:

- 1- Quais São as Categorias de Filmes Mais Comuns no IMDB?
- 2- Qual o Número de Títulos Por Gênero?
- 3- Qual a Mediana de Avaliação dos Filmes Por Gênero?
- 4- Qual a Mediana de Avaliação dos Filmes Em Relação ao Ano de Estréia?
- 5- Qual o Número de Filmes Avaliados Por Gênero Em Relação ao Ano de Estréia?
- 6- Qual o Filme Com Maior Tempo de Duração? Calcule os Percentis.
- 7- Qual a Relação Entre Duração e Gênero?
- 8- Qual o Número de Filmes Produzidos Por País?
- 9- Quais São os Top 10 Melhores Filmes?
- 10- Quais São os Top 10 Piores Filmes?

# Instalando e Carregando Pacotes

In [1]:
# Versão de  linguagem Python
from platform import python_version
print('Versão da Python usada Neste Jupyter Notebook:', python_version())

Versão da Python usada Neste Jupyter Notebook: 3.8.5


In [2]:
#instalar pacote
!pip install -q imdb-sqlite

In [4]:
#instalar o pacote com os paises.
!pip install -q pycountry

In [5]:
import re #pacete de expreção regulares em texto - processa texto
import time #para medir o tempo de exec ou trab com datas
import sqlite3 #pacote para manipular banco de dados
import pycountry#pacote com os paises
import numpy as np#manipulação de dados
import pandas as pd#manipulação de dados
import matplotlib.pyplot as plt #visualização de dados graficos
import seaborn as sns# visualização de dados graficos
from matplotlib import cm# visualização de dados graficos
from sklearn.feature_extraction.text import CountVectorizer#"skylearn" machine learnig python.
import warnings# filtra qualquer tipo warnings
warnings.filterwarnings("ignore")#filtra qualquer tipo de warnings
sns.set_theme(style ="whitegrid")#definição de estilo 'tema' Graficos

# Carregando os Dados
#### primeiro temos que baixa os dados!

In [ ]:
%%time
!imdb-sqlite

In [ ]:
#conectar no banco de dados
conn = sqlite3.connect("imdb.db")